## This example jupyer notebook uses clmm code to generate mock data with a set of configuration parameters, producing a catalog of ideal and noisy example data.

In [1]:
import numpy as np
import cluster_toolkit as ct
import matplotlib.pyplot as plt
%matplotlib inline

### Import mock data module and setup the configuration 

In [2]:
import sys
sys.path.append('./support')
sys.path.append('../clmm')
import modeling as pp
import mock_data as mock


### Mock data generation requires a config dictionary

In [3]:
config={}
config['cluster_m'] = 1.e15
config['cluster_z'] = 0.3
config['src_z'] = 0.8
config['concentration'] = 4

# If CCL is installed
# import pyccl as ccl
# config['cosmo'] = ccl.Cosmology(Omega_c=0.27, Omega_b=0.045, h=0.67, A_s=2.1e-9, n_s=0.96)

# If CCL is not installed use instead
from astropy.cosmology import FlatLambdaCDM
astropy_cosmology_object = FlatLambdaCDM(H0=70, Om0=0.27, Ob0=0.045)
config['cosmo'] = pp.cclify_astropy_cosmo(astropy_cosmology_object)

config['ngals'] = 10000
config['Delta'] = 200

# Instantiate the mock data objects
ideal_data = mock.MockData(config=config)
noisy_data = mock.MockData(config=config)
noisy_data_z = mock.MockData(config=config)

### Generate the mock catalog from the configuration. Consider 3 examples of increasing complexity:
* Ideal data, no noise, all galaxies at the same redshift
* Noisy data, including shape noise and redshift error, all galaxies at the same redshift
* Noisy data, galaxies following redshift distribution, redshift error, shape noise

In [4]:
ideal_data.generate()
noisy_data.generate(is_shapenoise=True, shapenoise=0.005, is_zerr=True)
noisy_data_z.generate(is_shapenoise=True, is_zdistribution=True, is_zerr=True)

#### Ideal catalog first entries: no noise on the shape measurement, all galaxies at z=0.3

In [5]:
ideal_data.catalog[0:3]

id,ra,dec,e1,e2,z
int64,float64,float64,float64,float64,float64
0,0.04891539840784862,-0.12838714512648997,0.011734726613294905,-0.010460257708536314,0.8
1,-0.02915469189917481,0.18906544224619926,0.010226648272935901,-0.003230810210346024,0.8
2,0.058969582625348724,0.13773656602341494,0.009850534312205594,0.010327731539882286,0.8


#### More realistic catalog: noise has been added on the shape measurement, galaxies follow some photoz distribution, gaussian pdf for each photoz is also provided (along with corresponding bins)

In [6]:
noisy_data.catalog[0:3]

id,ra,dec,e1,e2,z,z_pdf [34],z_bins [34]
int64,float64,float64,float64,float64,float64,float64,float64
0,-0.2106366954390113,-0.13417828021432837,-0.004721598235382816,0.010123341043769059,0.8,8.80221998747802e-07 .. 1.6217891916506561e-06,0.30000000000000004 .. 1.290000000000001
1,0.17207921870292125,-0.2143130128118167,0.0004257097571390653,-0.0019241196834748832,0.8,8.80221998747802e-07 .. 1.6217891916506561e-06,0.30000000000000004 .. 1.290000000000001
2,0.13448069709544125,0.06118801525325859,-0.011834257621229714,0.013580486639775935,0.8,8.80221998747802e-07 .. 1.6217891916506561e-06,0.30000000000000004 .. 1.290000000000001


In [ ]:
noisy_data_z.catalog[0:3]

In [ ]:
# Histogram of the redshift distribution of bkg galaxies (starting at z_cluster + 0.1)
hist = plt.hist(noisy_data_z.catalog['z'], bins=50)

In [ ]:
# pdz for the first galaxy in the catalog
plt.plot(noisy_data_z.catalog['z_bins'][0],noisy_data_z.catalog['z_pdf'][0])

### Compute and plot shear profile

This needs to be adapted to the new data profile making module

In [ ]:
# from clmm import ShearAzimuthalAverager

In [ ]:
# cl_dict = {'z':config['cluster_z'], 'ra':0.0, 'dec': 0.0}

# saa_ideal = ShearAzimuthalAverager(cl_dict,ideal_data.catalog)
# saa_noisy = ShearAzimuthalAverager(cl_dict,noisy_data.catalog)
# saa_noisy_z = ShearAzimuthalAverager(cl_dict,noisy_data_z.catalog)


# saa_ideal.compute_shear()
# saa_noisy.compute_shear()
# saa_noisy_z.compute_shear()

# saa_ideal.make_shear_profile()
# saa_noisy.make_shear_profile()
# saa_noisy_z.make_shear_profile()

In [ ]:
# Ideal data
# saa_ideal.plot_profile()

In [ ]:
# Noisy data, all galaxies at the same redshift
# saa_noisy.plot_profile()

In [ ]:
# Noisy data, galaxies following redshift distribution
# saa_noisy_z.plot_profile()